# Basic simulation of electrodes in ESPResSo part II:
# Electrolyte capacitor and Poisson--Boltzmann theory

In this tutorial we are going to investigate [...] using **ESPResSo**. 

To work with this tutorial, you should be familiar with the following topics:

1. Setting up and running simulations in ESPResSo - creating particles,
   incorporating interactions.
   If you are unfamiliar with this, you can go through the respective tutorial
   in the `lennard_jones` folder.
2. Basic knowledge of classical electrostatics:
   Dipoles, surface and image charges
3. Reduced units, as described in the ESPResSo user guide
   https://espressomd.github.io/doc/introduction.html#on-units

## Introduction

Understanding the electric double layer (EDL) is crucial for the study
of a variety of systems, including colloidal suspensions, charged biological
macromolecules and membranes. Poisson-Boltzmann (PB) theory based on a mean-field formalism properly describes the behavior of Coulomb fluids composed of monovalent ions at low concentrations in the
vicinity of weakly charged interfaces. However, for strongly charged systems, where correlation and finite size effects begin to dominate the system dynamics, the PB theory falls inadequate. Our goal in this module is to demonstrate how coarse grained implicit solvent simualtions can corroborate some of these approximate theories.

Here, we look at the ion density profile between two dielectric walls with constant potential applied to them. 
In this case the inclusion of dielectric inhomogeneities demands a detailed calculation of the image
effects that involve the full solution of the Poisson equation on the fly. This is dealt in a computational cost effective way using ELCIC method, to treat the image charge effect in the presence of 2D dielectric bounding
interfaces.  

[...]

## Theoretical Background 

[...]

In ELC-IC, the applied voltage between the electrodes consists of two
contributions,
$$ \Delta \phi_\mathrm{applied} = \Delta \phi_\mathrm{ind} + \phi_\mathrm{bat},$$
where $\Delta \phi_\mathrm{ind}$ denotes the potential drop due to the
polarization between the parallel electrodes (that is exactly counterbalanced by
the induced image charges).
In the constant potential ensemble, we obtain the target potential 
$\Delta \phi_\mathrm{bat}$ by superposing a homogeneous electric field created by
the induced charges with the corresponding battery voltage.
The contribution $\Delta \phi_\mathrm{ind}$ is calculated on the fly basically
for since the dipole moment $P_z$ is calculated in ELC anyway.
Let us explicitely calculate these two contributions (in reduced units):

## 1. System setup 

First we import all ESPResSo features and external modules

In [ ]:
import espressomd
import numpy as np
import espressomd.electrostatics
import espressomd.electrostatic_extensions
from espressomd.interactions import *
import espressomd.observables
import espressomd.accumulators
from espressomd import shapes

espressomd.assert_features(['WCA', 'ELECTROSTATICS'])

import matplotlib.pyplot as plt
from scipy.special import *
from scipy import constants as const
from tqdm import tqdm
from scipy.stats import sem

We need to define system dimensions and some physical parameters related to
length, time and energy scales of our system.
As discussed in previous tutorials, all physical parameters are defined in terms
of a length $\sigma$, mass $m$ and time $t$ and unit of charge $q$.
Since we are not explicitly interested in the dynamics of the system, we set the
mass to $m=1$ (Particle mass) and time $t=0.01 \tau$.
For convenience, we choose the elementary charge as fundamental unit ($q=1e$)
and $\sigma = 1 \,\mathrm{nm}$.

With this we can now define the fundamental parameters of our system:

In [ ]:
# water at room temperature
EPSILON_R = 78.4             # Relative dielectric constant of the water
TEMPERATURE = 300.0          # Temperature in Kelvin
BJERRUM_LENGTH = const.elementary_charge**2 / (4*np.pi*const.epsilon_0*EPSILON_R*const.Boltzmann*TEMPERATURE) / const.nano
# BERRUM_LENGTH of water at room temperature is 0.71 nm; electrostatic prefactor passed to P3M KBT/e2                

#Lennard-Jones  Parameters
LJ_SIGMA = 0.3              # Particle size nanometers, not point-like
LJ_EPSILON = 1.0
HS_ION_SIZE = 2**(1/6) * LJ_SIGMA

CONCENTRATION = 1e-2 # desired concentration 10 mmol/l
DISTANCE = 10 # 10 Debye lengths
N_IONPAIRS = 500

POTENTIAL_DIFF = 5.0

# Elementary charge 
q = np.array([1.0])  
types = {"Cation": 0, "Anion": 1  ,"Electrodes": 2}
charges = {"Cation": q[0], "Anion": -q[0]  }

### 1.1 Setting up the box dimensions and create system

We want to make use of the optimal performance of **ESPResSo** in this tutorial,
which is roughly at 1000 particles/core.
Thus, we fixed above the number of ion pairs to `N_IONPAIRS = 500`.

To be able to employ the analytical solution for the single plate also for the
double layer capacitor setup, the two electrodes need to be sufficiently far
away such that the additivity of the two surface potentials holds. In practice,
a separation of $d=10\lambda_\mathrm{D}$ is a good choice, represented by 
`DISTANCE = 10`.

Our choice of $c=10\,\mathrm{mmol}$ is a compromise between a sufficiently low
concentration for the PB theory to hold and not too large distances $d$ such
that the equilibration/diffusion of the ions takes too long
(`CONCENTRATION = 1e-2`).

Note that in order to obtain results that we can interpret easily, we explicitly
set a unit system using nanometers as length-scale above.
The corresponding ion size $\sigma$ (`HS_ION_SIZE`) of about 0.33 nm is a
typical value for a simple salt; this, however, is in sharp contrast to the
mean-field assumption of point-like ions.
The latter are not easily studied within Molecular Dynamics simulations due to
the required small time steps and are better suited for Monte-Carlo type
simulations.
We instead focus here on analyzing deviations from PB theory due to the finite
ion size.

The first task now is to write a function 
`get_box_dimension(concentration, distance, n_ionpairs=N_IONPAIRS)`
that returns the lateral and normal box lengths `box_l_xy` and `box_l_z` (in
nanometers) for the given parameters.

**Hint:** To account for the finite ion size and the wall interaction it is
useful to define the effective separation $d^\prime = d-2\sigma$, such that the
concentration is $\rho = N/(A*d^\prime)$.


```python
def get_box_dimension(concentration, distance, n_ionpairs=N_IONPAIRS):
    """ For a given number of particles, determine the lateral area of the box
    to match the desired concentration """

    # concentration is in mol/l, convert to 1/sigma**3
    rho = concentration * (const.Avogadro / const.liter) * const.nano**3
    debye_length = (4 * np.pi * BJERRUM_LENGTH * rho*2)**(-1./2) # desired Debye length in nm
    l_z = distance * debye_length
    
    box_volume = n_ionpairs / rho
    area = box_volume / (l_z - 2*HS_ION_SIZE) # account for finite ion size in density calculation
    l_xy = np.sqrt(area)

    return l_xy, l_z
```

In [ ]:
box_l_xy, box_l_z = get_box_dimension(CONCENTRATION,DISTANCE,N_IONPAIRS)

# useful quantities for the following calculations
DEBYE_LENGTH = box_l_z / DISTANCE # in units of nm
rho = N_IONPAIRS / (box_l_xy*box_l_xy*box_l_z) # in units of 1/nm^3

We now can create the **ESPResSo** system.

Note that for ELC to work properly, we need to add a gap of `ELC_GAP` in the
non-periodic direction.
The precise value highly affects the performance due to the tuning of the P3M
electrostatic solver.
For $d=10\lambda$ `ELC_GAP = 6*box_l_z` is a good value.

We also set the time-step $dt = 0.01 \tau$, which is limited by the choice of
$\sigma$ and $\tau$ in the repulsive WCA interaction.

In [ ]:
ELC_GAP = 6*box_l_z
system = espressomd.System(box_l=[box_l_xy, box_l_xy, box_l_z+ELC_GAP])
system.time_step = 0.01

### 1.2 Set up the double-layer capacitor

We now set up an electrolyte solution made of monovalent cations and anions
between two metallic electrodes at constant potential. 

#### 1.2.1 Electrode walls 

First, we add two wall constraints at $z=0$ and $z=L_z$ to stop particles from
crossing the boundaries and model the electrodes.

Refer to 
[espressomd.constraints.ShapeBasedConstraint](https://espressomd.github.io/doc/espressomd.html#espressomd.constraints.ShapeBasedConstraint)
and its
[wall constraint](https://espressomd.github.io/doc/constraints.html?highlight=constraint#wall)
in the documentation to set up constraints and the `types` dictionary for the
particle type.

```python
# Bottom wall, normal pointing in the +z direction 
floor = espressomd.shapes.Wall(normal=[0, 0, 1])
c1 = system.constraints.add(
    particle_type=types["Electrodes"], penetrable=False, shape=floor)

# Top wall, normal pointing in the -z direction
ceil = espressomd.shapes.Wall(normal=[0, 0, -1],
                              dist=-box_l_z)   
c2 = system.constraints.add(
    particle_type=types["Electrodes"], penetrable=False, shape=ceil)
```

### 1.2.2 Add particles for the ions

Now, place the ion pairs at random positions between the electrodes.

Note, that unfavorable overlap can be avoided by placing the particles in the
interval $[\sigma, d-\sigma]$ in the $z$-direction only.

``` python
offset=HS_ION_SIZE # avoid unfavorable overlap at close to the walls
Init_part_btw_z1=0+offset 
Init_part_btw_z2=box_l_z-offset
ion_pos=np.empty((3),dtype=float)

for i in range (N_IONPAIRS):
    ion_pos[0] = np.random.random(1) * system.box_l[0]
    ion_pos[1] = np.random.random(1) * system.box_l[1]
    ion_pos[2] = np.random.random(1) * (Init_part_btw_z2-Init_part_btw_z1) + Init_part_btw_z1
    system.part.add(pos=ion_pos, type=types["Cation"]  , q=charges["Cation"])
    
for i in range (N_IONPAIRS):
    ion_pos[0] = np.random.random(1) * system.box_l[0]
    ion_pos[1] = np.random.random(1) * system.box_l[1]
    ion_pos[2] = np.random.random(1) * (Init_part_btw_z2-Init_part_btw_z1) + Init_part_btw_z1
    system.part.add(pos=ion_pos, type=types["Anion"]  , q=charges["Anion"])
```

### 1.2.3 Add interactions:
For excluded volume interactions, we add a WCA potential. 

Refer to the documentation to set up the
[WCA interaction](https://espressomd.github.io/doc/espressomd.html#espressomd.interactions.WCAInteraction) 
under [Non-bonded](https://espressomd.github.io/doc/inter_non-bonded.html)
section.



``` python
for  key, val in types.items():
    for key1, val1 in types.items():
        system.non_bonded_inter[val, val1].wca.set_params(epsilon=LJ_EPSILON, sigma=LJ_SIGMA)
```

For the (2D+h) electrostatic with dielectrics we choose the ELC-IC with P3M.

Refer the documentation to set up
[ELCIC with P3M](https://espressomd.github.io/doc/electrostatics.html#electrostatic-layer-correction-elc)
under the [electrostatics](https://espressomd.github.io/doc/electrostatics.html)
section. 

As later we will study different potential drops between the electrodes, write a
function that sets up the electrostatic solver for a given value
`POTENTIAL_DIFF.`
This function will take care of tuning the P3M and ELC parameters.
For our purposes, an accuracy of $10^{-3}$ is sufficient.

The function should have a signature
`setup_electrostatic_solver(potential_diff)` and return the ELC instance.

``` python
def setup_electrostatic_solver(potential_diff):

    delta_mid_top = -1.0   #(Fully metallic case both -1)                 
    delta_mid_bot = -1.0

    accuracy = 1e-3
    check_accuracy = 1e-3
    p3m = espressomd.electrostatics.P3M(prefactor=BJERRUM_LENGTH,
                                        accuracy=accuracy, 
                                        tune=True,
                                        )
    
    elc = espressomd.electrostatics.ELC(actor=p3m,
                                        gap_size=ELC_GAP,
                                        const_pot=True,
                                        pot_diff=potential_diff,
                                        maxPWerror=check_accuracy,
                                        delta_mid_bot=delta_mid_bot,
                                        delta_mid_top=delta_mid_top)
    
    return elc
```

Now add the solver to the system:

In [ ]:
system.electrostatics.solver = setup_electrostatic_solver(POTENTIAL_DIFF)

## 2. Equilibration

### 2.1 Steepest descent

Before we can start the simulation, we need to remove the overlap between
particles to avoid large forces which would crash the simulation.

For this, we use the steepest descent integrator with a relative convergence
criterion for forces and energies.

After steepest descent, we switch to a Velocity Verlet integrator and set up a
Langevin thermostat.
Note, that we only analyze static properties, thus the damping and temperature
chosen here only determine the relaxation speed towards the equilibrium
distribution.

In [ ]:
#  Relax the overlaps with steepest descent

system.integrator.set_steepest_descent(f_max=10, gamma=50.0,
                                       max_displacement=0.02)
system.integrator.run(1000)
system.integrator.set_vv() # Switch bach to Velocity Verlet 

# Add thermostat 
thermostat_seed = np.random.randint(np.random.randint(1000000))
system.thermostat.set_langevin(kT=1.0, gamma=0.1, seed=thermostat_seed)

## Equilibrate the ion distribution

Convergence after $t\sim25$ time units, possible to run up to $t=100$ here...
This is a total of 10.000 time steps (~1 minute).

In [ ]:
# Equlibration parameters
STEPS_PER_SAMPLE = 200
N_SAMPLES = 50
N_PART = 2* N_IONPAIRS

times = np.zeros(N_SAMPLES)
e_total = np.zeros_like(times)
e_kin = np.zeros_like(times)

for i in tqdm(range(N_SAMPLES)):
    times[i] = system.time
    energy = system.analysis.energy()
    e_total[i] = energy['total']
    e_kin[i] = energy['kinetic']
    system.integrator.run(STEPS_PER_SAMPLE)

In [ ]:
# Plot the convergence of the total energy
plt.figure(figsize=(10, 6))
plt.plot(times, e_total, label='total')
plt.plot(times, e_kin, label='kinetic')
plt.xlabel('t')
plt.ylabel('E')
plt.legend()
plt.show()

## 3. Calculate and analyze ion profile

### 3.1 Set up the density accumulators

We now need to set up an 
[espressomd.observables.DensityProfile](https://espressomd.github.io/doc/espressomd.html#espressomd.observables.DensityProfile)
observable to calculate the anion and cation density profiles.

The time average is obtained through a
[espressomd.accumulators.MeanVarianceCalculator](espressomd.accumulators.MeanVarianceCalculator).

Write a function `setup_densityprofile_accumulators(bin_width)` that returns the
`bin_centers` and the accumulators for both ion species in the $z$-range $[0,d]$.
Since we are not estimating errors in this tutorial, the choice of `delta_N` is
rather arbitrary and does not affect the results. In practice, a typical value is
`delta_N=20`.

```python
def setup_densityprofile_accumulators(bin_width):

    Ion_id=[]
    Cations = system.part.select(type=types["Cation"])
    Cations_id=[]
    for i in Cations:
        Cations_id.append(i.id)
        Ion_id.append(i.id)
        
    Anions = system.part.select(type=types["Anion"])
    Anions_id=[]
    for i in Anions:
        Anions_id.append(i.id)
        Ion_id.append(i.id)
    
    n_z_bins = int(np.round((system.box_l[2] - ELC_GAP) / bin_width))
    
    # Accumulator 1 : observable::Density_Profile
    density_profile_cation = espressomd.observables.DensityProfile(ids=Cations_id,
                                                           n_x_bins=1,
                                                           n_y_bins=1,
                                                           n_z_bins=n_z_bins,
                                                           min_x=0,
                                                           min_y=0,
                                                           min_z=0,
                                                           max_x=system.box_l[0],
                                                           max_y=system.box_l[1],
                                                           max_z=system.box_l[2] - ELC_GAP)
    
    density_accumulator_cation = espressomd.accumulators.MeanVarianceCalculator(obs=density_profile_cation, delta_N=20)
    
    
    density_profile_anion = espressomd.observables.DensityProfile(ids=Anions_id,
                                                           n_x_bins=1,
                                                           n_y_bins=1,
                                                           n_z_bins=n_z_bins,
                                                           min_x=0,
                                                           min_y=0,
                                                           min_z=0,
                                                           max_x=system.box_l[0],
                                                           max_y=system.box_l[1],
                                                           max_z=system.box_l[2] - ELC_GAP)
    
    density_accumulator_anion = espressomd.accumulators.MeanVarianceCalculator(obs=density_profile_anion, delta_N=20)

    zs = density_profile_anion.bin_centers()[0, 0, :, 2]
    
    return zs, density_accumulator_cation, density_accumulator_anion
```

In [ ]:
zs, density_accumulator_cation, density_accumulator_anion = setup_densityprofile_accumulators(bin_width = DEBYE_LENGTH/10.)

### 3.2 Run the simulation

Now we take some measurement sampling the density profiles.

In [ ]:
N_SAMPLES = 25

# Add the accumulators
system.auto_update_accumulators.clear()
system.auto_update_accumulators.add(density_accumulator_cation)
system.auto_update_accumulators.add(density_accumulator_anion)
    
times=[]
e_total=[]
for tm in tqdm(range(N_SAMPLES)):
    system.integrator.run(STEPS_PER_SAMPLE)
    times.append( system.time)
    energy = system.analysis.energy()
    e_total.append( energy['total'])   

cation_profile_mean = density_accumulator_cation.mean()[0, 0, :]
anion_profile_mean = density_accumulator_anion.mean()[0, 0, :]

### Compare to analytical prediction

Since we assume pair-wise additivity, the total ion density follows from
$$ \rho (z) = \rho_+(z) - \rho_+ (d-z) + \rho_-(z) - \rho_-(d-z) .$$

In [ ]:
def gouy_chapman_potential(x, debye_length, phi_0):
    kappa = 1./debye_length
    return 2*np.log((1 + np.tanh(1./4*(phi_0) * np.exp(-kappa*x))) \
                  / (1 - np.tanh(1./4*(phi_0) * np.exp(-kappa*x))))

def gouy_chapman_density(x, c0, debye_length, phi_0):
    phi = gouy_chapman_potential(x, debye_length, phi_0)
    return c0/2. * np.exp(-phi)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(zs, cation_profile_mean, color='b', label='cation')
plt.plot(zs, anion_profile_mean, color='r', label='anion')
plt.plot(zs, cation_profile_mean + anion_profile_mean, color='k', label='total')

x = np.linspace(HS_ION_SIZE, box_l_z-HS_ION_SIZE, 100)
plt.plot(x, (gouy_chapman_density(x, CONCENTRATION, DEBYE_LENGTH,-POTENTIAL_DIFF/2.) \
         + gouy_chapman_density(box_l_z-HS_ION_SIZE-x, CONCENTRATION, DEBYE_LENGTH,POTENTIAL_DIFF/2.))/2., color='r', ls='--')
plt.plot(x, (gouy_chapman_density(box_l_z-HS_ION_SIZE-x, CONCENTRATION, DEBYE_LENGTH,-POTENTIAL_DIFF/2.) \
         + gouy_chapman_density(x, CONCENTRATION, DEBYE_LENGTH,POTENTIAL_DIFF/2.) )/2., color='b', ls='--')
plt.plot(x, (gouy_chapman_density(x, CONCENTRATION, DEBYE_LENGTH,-POTENTIAL_DIFF/2.) \
         + gouy_chapman_density(box_l_z-HS_ION_SIZE-x, CONCENTRATION, DEBYE_LENGTH,POTENTIAL_DIFF/2.))/2 \
         + (gouy_chapman_density(box_l_z-HS_ION_SIZE-x, CONCENTRATION, DEBYE_LENGTH,-POTENTIAL_DIFF/2.) \
         + gouy_chapman_density(x, CONCENTRATION, DEBYE_LENGTH,POTENTIAL_DIFF/2.) )/2., color='k', ls='--', lw=2)

plt.legend()
plt.xlabel(r'$z\,\mathrm{[nm]}$')
plt.ylabel(r'$\rho(z)\,\mathrm{[mol/l]}$')
plt.show()

We now check how well the surface charge agrees with Graham equation.
To this end we calculate 
$$\sigma = \int_0^{d/2} \rho(z) \,\mathrm{d}z .$$

In [ ]:
#Test sigma with Graham equation
sigma_left = np.sum((cation_profile_mean-anion_profile_mean)[:int(len(zs)/2.)]) * (zs[1]-zs[0])
sigma_right = np.sum((+cation_profile_mean-anion_profile_mean)[int(len(zs)/2.):]) * (zs[1]-zs[0])

def graham_sigma(phi):
    return np.sinh(phi/4.) * np.sqrt(2*rho/(np.pi*BJERRUM_LENGTH))
sigma_graham = graham_sigma(POTENTIAL_DIFF)

# sigma in e/nm^2
print('simulation:', sigma_right, 'graham:', sigma_graham)


The electric field is readily obtained from the integral 
$$E(z) = \int_0^{z} \frac{1}{\epsilon_0 \epsilon_r} \rho(z^\prime) \,\mathrm{d}z^\prime .$$

In [ ]:
# plot the electric field
f, ax = plt.subplots(figsize=(10, 6))

dz_SI = (zs[1]-zs[0])*const.nano
chargedensity = (cation_profile_mean-anion_profile_mean)*const.elementary_charge/const.nano**3 
E_SI = 1/(EPSILON_R*const.epsilon_0)* np.cumsum(chargedensity*dz_SI)
# integration constant: zero field in the center
E_SI -= E_SI.min()
E = E_SI / (const.elementary_charge / (const.Boltzmann * TEMPERATURE) / const.nano)
ax2 = plt.twinx()

ax.plot(zs,E_SI)
ax2.plot(zs,E)
ax.set_xlabel(r'$z\,\mathrm{[nm]}$')
ax.set_ylabel(r'$E_\mathrm{ind}\,\mathrm{[V/m]}$')
ax2.set_ylabel(r'$E_\mathrm{ind}\,\mathrm{[(k_\mathrm{B}T/e)/nm]}$')
plt.show()

And the electric potential from $\phi(z) = \int_0^z -E(z^\prime)\,\mathrm{d}z^\prime$.

In [ ]:
# plot the elecrostatic potential
f, ax = plt.subplots(figsize=(10, 6))
ax2 = ax.twinx()
phi_SI = -np.cumsum(E_SI*dz_SI)
phi = phi_SI * (const.elementary_charge / (const.Boltzmann * TEMPERATURE))
ax.plot(zs, phi_SI)
ax2.plot(zs, phi)
ax.set_xlabel(r'$z\,\mathrm{[nm]}$')
ax.set_ylabel(r'$\phi\,[V]$')
ax2.set_ylabel(r'$\phi\,[k_\mathrm{B}T/e]$')
ax2.axhline(-5, ls='--', color='k')
ax.axhline(-5 / (const.elementary_charge / (const.Boltzmann * TEMPERATURE)))
ax2.axhline(0, ls='--', color='k')
ax.axhline(0 / (const.elementary_charge / (const.Boltzmann * TEMPERATURE)))
ax.set_xlim(0, 10*DEBYE_LENGTH)
plt.show()

In [ ]:
measured_potential_difference = -(phi[-1]+phi[0]),
print('applied voltage', POTENTIAL_DIFF, 'measured voltage', measured_potential_difference,
      'deviation:', measured_potential_difference/POTENTIAL_DIFF)

## 4. Differential capacitance

With the above knowledge, we can now easily assess the 
differential capacitance of the system, i.e. play the applied voltage difference
and determine the corresponding surface charge density.

In [ ]:
sigma_vs_phi = []
for potential_diff in tqdm(np.linspace(.5,14,10)):

    system.electrostatics.solver = setup_electrostatic_solver(potential_diff)

    N_EQUIL = 8
    N_SAMPLES = 8
    times=[]
    e_total=[]
    sigmas = []
    for tm in range(N_EQUIL):
        system.integrator.run(STEPS_PER_SAMPLE)
        times.append( system.time)
        energy = system.analysis.energy()
        e_total.append( energy['total'])   

    for tm in tqdm(range(N_SAMPLES)):

        zs, density_accumulator_cation, density_accumulator_anion = setup_densityprofile_accumulators(bin_width = DEBYE_LENGTH/10.)

        system.auto_update_accumulators.clear()
        system.auto_update_accumulators.add(density_accumulator_cation)
        system.auto_update_accumulators.add(density_accumulator_anion)

        system.integrator.run(STEPS_PER_SAMPLE)
        times.append( system.time)
        energy = system.analysis.energy()
        e_total.append( energy['total'])  

        cation_profile_mean = density_accumulator_cation.mean()[0, 0, :]
        anion_profile_mean = density_accumulator_anion.mean()[0, 0, :]

        sigmas.append(np.sum((cation_profile_mean-anion_profile_mean)[:int(len(zs)/2.)]) * (zs[1]-zs[0]))

    sigma_vs_phi.append([POTENTIAL_DIFF, np.mean(sigmas), sem(sigmas)]) 

In [ ]:
f, ax = plt.subplots(figsize=(10, 6))
x = np.linspace(0,7.5)
sigma_vs_phi = np.array(sigma_vs_phi)
phi_SI = sigma_vs_phi[:,0] / (const.elementary_charge / (const.Boltzmann * TEMPERATURE))
plt.errorbar(-sigma_vs_phi[:,1]*const.elementary_charge/const.nano**2, phi_SI, xerr=sigma_vs_phi[:,2]*const.elementary_charge/const.nano**2, fmt='o',label='Sim')
plt.plot(graham_sigma(x)*const.elementary_charge/const.nano**2,
        x / (const.elementary_charge / (const.Boltzmann * TEMPERATURE)), label='Graham')
x = np.linspace(0,0.3)
plt.plot(EPSILON_R*const.epsilon_0*x/2/(DEBYE_LENGTH*const.nano),x, label='linear PB', ls='--')
plt.xlabel(r'$\sigma\,\mathrm{[C/m^2]}$')
plt.ylabel(r'$\Psi_0\,\mathrm{[V]}$')
plt.legend()
plt.show()

## References

<a id='[1]'></a>[1] 
